### Import packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import LogLocator
from datetime import datetime
import os
import scienceplots

plt.style.use('science')

### Basic formulas

This part consists of the basic basic formulas that are used troughout the calculations:

- $\mathtt{alpha(n, p)}$: $x_3$ (perpendicular) component of the wave vector with a lateral wave vector $\vec{p}$ in a medium with a refractive index $n$ (which can be a complex number).

- $\mathtt{g(p, a)}$: Gaussian correlation function for given lateral wave vector $\vec{p}$ and correlation length parameter $a$.

- $\mathtt{es(q)}$: s-polarization unit vector for a lateral wave vector $\vec{q}$. In case of a normal incidence ($\vec{q}=0$), this unit vector is set to the $x_2$ direction.

- $\mathtt{ep(sign, ind, delta, beta, q)}$: p-polarization unit vector for a lateral wave vector $\vec{q}$ and $\mathtt{sign}$ direction of the perpendicular wave vector in medium $\mathtt{ind}$. In case of a normal incidence ($\vec{q}=0$), this unit vector is set to the $x_1$ direction.

- $\mathtt{M(b, a, l, m, delta, beta, p, q)}$: Coordinate system transformation matrix defined as in Eq. (30).

- $\mathtt{rho0(delta, beta, p0)}$: The zeroth order perturbative solution of the reflected wave amplitude matrix.

- $\mathtt{tau0(delta, beta, p0)}$: The zeroth order perturbative solution of the transmitted wave amplitude matrix.

- $\mathtt{rho1(delta, beta, p, p0)}$: The first order perturbative solution of the reflected wave amplitude matrix.

- $\mathtt{tau1(delta, beta, p, p0)}$: The first order perturbative solution of the transmitted wave amplitude matrix.

In [ ]:
def alpha(n, p):
    return np.sqrt(n**2 - np.linalg.norm(p)**2 + 0j)

def g(p, a):
    return np.pi * a**2 * np.exp(-np.pi**2 * np.linalg.norm(p)**2 * a**2)

def es(q):
    if (np.linalg.norm(q) == 0.0):
        return np.array([0.0, 1.0, 0.0])
    else:
        e3 = np.array([0.0, 0.0, 1.0])
        qhat = q / np.linalg.norm(q)

        return np.cross(e3, qhat)

def ep(sign, ind, delta, beta, q):
    n1 = 1
    n2 = 1 - delta - 1j*beta

    if (ind == 1):
        n = n1
    elif (ind == 2):
        n = n2
    else:
        return 0.0

    if (np.linalg.norm(q) == 0.0):
        return np.array([1.0, 0.0, 0.0])
    else:
        e3 = np.array([0.0, 0.0, 1.0])
        qhat = q / np.linalg.norm(q)

        if (sign == "+"):
            pm = 1.0
        elif (sign == "-"):
            pm = -1.0
        else:
            return 0.0

        return -1 * (pm * np.sqrt(n**2 - np.linalg.norm(q)**2 + 0j) * qhat - np.linalg.norm(q) * e3) / n


def M(b, a, l, m, delta, beta, p, q):
    n1 = 1
    n2 = 1 - delta - 1j*beta

    Mpq = np.zeros(shape=(2, 2), dtype=np.complex64)
    Mpq[0, 0] = np.dot(ep(b, l, delta, beta, p), ep(a, m, delta, beta, q))
    Mpq[0, 1] = np.dot(ep(b, l, delta, beta, p), es(q))
    Mpq[1, 0] = np.dot(es(p), ep(a, m, delta, beta, q))
    Mpq[1, 1] = np.dot(es(p), es(q))

    return n1 * n2 * Mpq


def rho0(delta, beta, p0):
    n1 = 1
    n2 = 1 - delta - 1j*beta
    
    x = (alpha(n1, p0) - alpha(n2, p0)) / (alpha(n2, p0) + alpha(n1, p0))
    M1 = M("+", "+", 2, 1, delta, beta, p0, p0)
    M2 = M("+", "-", 2, 1, delta, beta, p0, p0)

    return x * (np.linalg.inv(M1) @ M2)

def tau0(delta, beta, p0):
    n1 = 1
    n2 = 1 - delta - 1j*beta

    x = 2 * (n1 * n2 / (n2**2 - n1**2)) * alpha(n1, p0) * (alpha(n2, p0) - alpha(n2, p0))
    M1 = M("-", "-", 1, 2, delta, beta, p0, p0)

    return x * np.linalg.inv(M1)


def rho1(delta, beta, p, p0):
    n1 = 1
    n2 = 1 - delta - 1j*beta
    
    x = alpha(n1, p) - alpha(n2, p)
    M1 = M("+", "+", 2, 1, delta, beta, p, p)
    M2 = M("+", "-", 2, 1, delta, beta, p, p0)
    M3 = M("+", "+", 2, 1, delta, beta, p, p0)
    rho = rho0(delta, beta, p0)

    return x * np.linalg.inv(M1) @ (M2 + (M3 @ rho))

def tau1(delta, beta, p, p0):
    n1 = 1
    n2 = 1 - delta - 1j*beta
    
    x = alpha(n1, p) - alpha(n2, p)
    M1 = M("-", "-", 1, 2, delta, beta, p, p)
    M2 = M("-", "-", 1, 2, delta, beta, p, p0)
    tau = tau0(delta, beta, p0)

    return x * np.linalg.inv(M1) @ (M2 @ tau)

### Observables

There are two observables considered in this part. Both are assuming a coplanar scattering problem.

- $\mathtt{reflectivity(delta, beta, theta0)}$: Total reflection coefficient integrated for all possible forward scattering reflected angle as a function of the incident angle $\theta_0$, focusing on the coherent component of the reflected wave intensity.

- $\mathtt{MDRC(delta, beta, theta0, thetar, a, sigma)}$: Incoherent component of mean differential reflection coefficient (MDRC) as a function of the incident angle $\theta_0$ and reflected angle $\theta_r$ for an ensemble of randomly rough surface characterized by an rms rougness $\sigma$ and correlation length $a$.

In [ ]:
def reflectivity(delta, beta, theta0):
    ref = np.zeros(shape=(2, 2))

    p0 = np.array([np.cos(theta0), 0, 0])
    rho_0 = rho0(delta, beta, p0)
    ref[:, :] = np.abs(rho_0[:, :])**2

    return ref

def MDRC(delta, beta, theta0, thetar, a, sigma):
    n1 = 1
    ang = (2 * np.pi)**2 * n1**2 * (np.sin(thetar) / np.sin(theta0))**2

    p0 = n1 * np.array([np.cos(theta0), 0, 0])
    p = n1 * np.array([np.cos(thetar), 0, 0])
    dp = p - p0

    R = np.zeros(shape=(2, 2))
    rho = rho1(delta, beta, p, p0)

    R[:, :] = np.abs(rho[:, :])**2

    return ang * sigma**2 * g(dp, a) * R

### Critical angle calculation

$\mathtt{thetacr(sample, maxscan, scanpoints)}$ calculates the critical angle of a material in which its refracted index has been retrieved from the CXRO website in a file name format $\mathtt{sample.dat}$. The critical angle is calculated for all energy values in the provided refractive index data. For each energy value, the Snell's law critical angle is first evaluated using the real component of the refractive index. The reflectivity-based critical angle is then calculated with the reflectivy value is scanned up to $\mathtt{maxscan}$ times of the Snell's law critical angle with $\mathtt{scanpoints}$ number of points until a half value is obtained. Once the calculation is finisihed, the results are written in a txt file and both Snell's law and reflectivity-based critical angle are plotted as a function of the incident photon energy. The txt file will be the main input file to retrieve the requied information for the rest of the calculations. Therefore, this critical angle calculation must be performed at least once before proceeding to the next calculations.

In [ ]:
def theta_cr(sample, maxscan, scanpoints):
    
    outdir = sample + "/"
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    summary_output = os.path.join(outdir, "critical_angle.txt")

    filename = sample + ".dat"
    data = np.loadtxt(filename, skiprows=2)
    energies = data[:, 0]
    deltas = data[:, 1]
    betas = data[:, 2]
    
    date_start = datetime.now()
    
    points = len(energies)
    thetac_th = np.zeros(points)
    thetac_exp = np.zeros(points)

    with open(summary_output, 'w'):
        pass
        
    with open(summary_output, 'a') as file:
        file.write("Date started : " + date_start.strftime("%d-%m-%Y_%H.%M.%S") + "\n")
        file.write("Sample : "+ sample + " \n")
        file.write("\n")
        file.write("energy [eV] - \t δ - \t β - \t θc_sn [rad] - \t θc_rf [rad] \n")
        file.write("\n")

    for i in range(points):
        energy = energies[i]
        delta = deltas[i]
        beta = betas[i]

        if delta > 0.0:
            thc_theory = np.sqrt(2*delta)
            thetac_th[i] = thc_theory
            theta_i = thc_theory * np.linspace(0, maxscan, num=scanpoints)

            for j in range(scanpoints):
                ref = reflectivity(delta, beta, theta_i[j])
                refss = ref[1, 1]

                if refss <= 0.5:
                    thc_experiment = theta_i[j]
                    thetac_exp[i] = thc_experiment
                    break
        else:
            thc_experiment = thc_theory = 0.0

        with open(summary_output, 'a') as file:
            file.write(str(energy) + " \t " + str(delta) + " \t " + str(beta) +
                       " \t " + str(thc_theory) + " \t " + str(thc_experiment) + "\n")
    
    #################################################################################################################################

    plt.figure(figsize=(3.375, 2.5)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "Critical Angle"    
    plt.plot(energies, thetac_exp, linewidth=1.5, color="green", label="reflectivity")
    plt.plot(energies, thetac_th, linewidth=1.5, color="red", linestyle="dashed", label="Snell")
    plt.xscale('log')
            
    plt.xlabel('energy [eV]')
    plt.ylabel('critical angle [rad]')

    plt.legend(loc='best')
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    plt.figure(figsize=(3.375, 2.5)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()
    
    title = r"Refraction Index $(1-\delta)-i\beta$" 
    plt.title(title)
    plt.plot(energies, deltas, linewidth=1.5, color="green", label=r"$\delta$")
    plt.plot(energies, betas, linewidth=1.5, color="red", linestyle='dashed', label=r"$\beta$")
    plt.xscale('log')
    plt.yscale('log')
    plt.gca().yaxis.set_major_locator(LogLocator(base=10.0, subs=[1.0], numticks=10))
            
    plt.xlabel('energy [eV]')
    plt.ylabel(r'$\delta$, $\beta$')

    plt.legend(loc='best')
    plt.grid(True)

    plot_title = "Refraction Index.pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    date_finish = datetime.now()
    finish = date_finish.strftime("%d-%m-%Y_%H.%M.%S")
    with open(summary_output, 'a') as file:
        file.write("\n")
        file.write("Date finished : " + finish + "\n")                         

### Output file reader

This function reads the critical angle txt output file that contains the information needed for reflectivity and MDRC calculations.

In [5]:
def readfile(sample, energy):
    outdir = sample + "/"
    if not os.path.exists(outdir):
        raise FileNotFoundError("File not found")
    
    found = False
    filename = os.path.join(outdir, "critical_angle.txt")

    with open(filename, "r") as file:
        for _ in range(5):
            next(file)
        
        for line in file:
            parts = line.strip().split()
            if parts[0] == str(energy):
                delta = float(parts[1])
                beta = float(parts[2])
                thetac_sn = float(parts[3])
                thetac_rf = float(parts[4])
                found = True
                break
    
    if not found:
        raise LookupError("Energy not found")
    
    return delta, beta, thetac_sn, thetac_rf

### Reflectivity calculation

$\mathtt{refplot(sample, energy, points, thetamax, usebeta)}$ produces the reflectiviy plot of the $\mathtt{sample}$ at the given incident photon $\mathtt{energy}$ value, focusing on the s to s and p to p polarization reflection problems. The incident angle range is from zero up to $\mathtt{thetamax}$ times of the critical angle calculated based on the half-valued reflectivity point with $\mathtt{points}$ number of data points. $\mathtt{usebeta}$ can be used to switch between the realistic and the absorption-less case. Setting $\mathtt{usebeta}=\mathtt{False}$ forces the imaginary component of refractive index into a zero value, allowing us to analyze the role of absoprtion in the reflectivity plot. All calculation results are written in a txt file, and the resulting s-s and p-p reflectivity curve are plotted in a single figure.

In [ ]:
def ref_plot(sample, energy, points, thetamax, use_beta):
    date_start = datetime.now()

    outdir = sample + "/reflectivity/" + str(energy) + " eV/0-" + str(np.round(thetamax, 1)) + " θc"

    delta, beta, thetac_sn, thetac_rf = readfile(sample, energy)

    if not use_beta:
        beta = 0.0
        outdir = sample + "/reflectivity/" + str(energy) + " eV/0-" + str(np.round(thetamax, 1)) + " θc (without β)"

    thetac = thetac_rf
    
    if not os.path.exists(outdir):
        os.makedirs(outdir)
    summary_output = os.path.join(outdir, "reflectivity.txt")

    theta_i = thetac * np.linspace(0, thetamax, num=points)
    ref_ss = np.zeros(points)
    ref_pp = np.zeros(points)

    with open(summary_output, 'w'):
        pass
        
    with open(summary_output, 'a') as file:
        file.write("Date started : " + date_start.strftime("%d-%m-%Y_%H.%M.%S") + "\n")
        file.write("Sample : " + str(sample) + "\n")
        file.write("X-ray energy (E) : " + str(energy) + " eV \n")
        file.write("δ : " + str(delta) + "\n")
        file.write("β : " + str(beta) + "\n")
        file.write("\n")
        file.write("θi [rad] - \t R_ss - \t R_pp \n")
        file.write("\n")
    
    for i in range(points):
        ref = reflectivity(delta, beta, theta_i[i])
        ref_ss[i] = ref[1, 1]
        ref_pp[i] = ref[0, 0]

        with open(summary_output, 'a') as file:
            file.write(str(theta_i[i]) + " \t " + str(ref_ss[i]) + " \t " + str(ref_pp[i]) +"\n")
    
    #################################################################################################################################

    plt.figure(figsize=(3.375, 2.5)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "Reflectivity"    
    plt.plot(theta_i, ref_ss, linewidth=1.5, color="green", label="s-s")
    plt.plot(theta_i, ref_pp, linewidth=1.5, color="red", linestyle="dashed", label="p-p")
    plt.axhline(y=0.5, linewidth=0.5, color="black")
            
    plt.xlabel('incidence angle [rad]')
    plt.ylabel('reflectivity')

    plt.legend(loc='best', title="polarization")
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    date_finish = datetime.now()
    finish = date_finish.strftime("%d-%m-%Y_%H.%M.%S")
    with open(summary_output, 'a') as file:
        file.write("\n")
        file.write("Date finished : " + finish + "\n")

### MDRC and its first derivative calculations

$\mathtt{mdrc(sample, energy, alength, sigma, points, thetai, thetarmax, crmode, usebeta)}$ is the main function of this program. It calculates the incoherent component of MDRC alongside its first derivative with respect to the reflected angle for the s to s and p to p polarization reflection problems of the $\mathtt{sample}$ at a particular incident light $\mathtt{energy}$. The interface between two mediums is represented by an ensemble of randomly rough surface with a correlation length $\mathtt{alength}$ and rms roughness $\mathtt{sigma}$. User may put an array of the incident angle $\mathtt{thetai}$ values and for each incident angle, the reflected angle is set from zero up to $\mathtt{thetarmax}$ times of the critical angle. The critical angle definition can be either based on $\mathtt{crmode}=\mathtt{"snell"}$ or $\mathtt{crmode}=\mathtt{"reflectivity"}$. Similarly as in the reflectiviy calculation, $\mathtt{usebeta}$ can be set to $\mathtt{False}$ to study the effect of neglecting absoprtion. All calculation results are written in a txt file, and the resulting MDRC and its first derivative are plotted in a single figure for all given incident angle values.

In [ ]:
def mdrc(sample, energy, alength, sigma, points, thetai, thetarmax, cr_mode, use_beta):
    date_start = datetime.now()

    outdir = sample + "/mdrc/" + str(energy) + " eV/a " + str(alength) + "λ/θi " + str(thetai[0]) + "θc-" + str(thetai[-1]) + "θc" 

    delta, beta, thetac_sn, thetac_rf = readfile(sample, energy)

    if not use_beta:
        beta = 0.0
        outdir = sample + "/mdrc/" + str(energy) + " eV/a " + str(alength) + "λ/θi " + str(thetai[0]) + "θc-" + str(thetai[-1]) + "θc (without β)"

    if not os.path.exists(outdir):
        os.makedirs(outdir)
    summary_output = os.path.join(outdir, "summary.txt")

    wvlength = 1240/energy

    if cr_mode == "snell":
        thetac = thetac_sn
    elif cr_mode == "reflectivity":
        thetac = thetac_rf
    else:
        raise ValueError("Please use either 'snell' or 'reflectivity'")
    
    theta_inc = thetac * np.array(thetai)
    theta_r = thetac * np.linspace(0, thetarmax, num=points)
    dtheta = theta_r[1] - theta_r[0]

    with open(summary_output, 'w'):
        pass

    with open(summary_output, 'a') as file:
        file.write("Date started : " + date_start.strftime("%d-%m-%Y_%H.%M.%S") + " \n")
        file.write("\n")
        file.write("Sample : " + str(sample) + "\n")
        file.write("\n")
        file.write("X-ray energy (E) : " + str(energy) + " eV \n")
        file.write("X-ray wavelength (λ) : " + str(np.round(wvlength, 3)) + " nm \n")
        file.write("δ : " + str(delta) + "\n")
        file.write("β : " + str(beta) + "\n")
        file.write("Critical angle : " + str(thetac) + " rad \n")
        file.write("\n")
        file.write("Correlation length : " + str(alength) + "λ \n")
        file.write("RMS roughness : " + str(sigma) + "λ \n")
        file.write("\n")
        file.write("θi [rad] \t θr [rad] \t mdrc (s-s) \t dmdrc (s-s) \t mdrc (p-p) \t dmdrc (p-p) \n")
        file.write("\n")
    
    MDRC_list = np.zeros(shape=(2, 2, len(theta_inc), points))
    dMDRC_list = np.zeros(shape=(2, 2, len(theta_inc), points))

    for i in range(len(theta_inc)):
        mdrc0 = np.zeros(shape=(2, 2))
        mdrc1 = np.zeros(shape=(2, 2))
        
        for j in range(points):
            MDRC_list[:, :, i, j] = MDRC(delta, beta, theta_inc[i], theta_r[j], alength, sigma)
            mdrc1[:, :] = MDRC_list[:, :, i, j]

            dMDRC_list[:, :, i, j] = (mdrc1[:, :] - mdrc0[:, :]) / dtheta
            mdrc0[:, :] = mdrc1[:, :]

            with open(summary_output, 'a') as file:
                file.write(str(theta_inc[i]) + " \t " + str(theta_r[j]) + " \t " + str(MDRC_list[1, 1, i, j]) + 
                           " \t " + str(dMDRC_list[1, 1, i, j]) + " \t " + str(MDRC_list[0, 0, i, j]) + " \t " +
                           str(dMDRC_list[0, 0, i, j]) + "\n") 

    #################################################################################################################################
    
    plt.figure(figsize=(3.375, 3.0)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "MDRC s-s Polarization"
    mdrc = np.zeros(shape=points)
    line_style = ['-.', '--', '-', '--', '-.']

    for i in range(len(theta_inc)):
        mdrc[:] = MDRC_list[1, 1, i, :]
        label = str(np.round(theta_inc[i]/thetac, 2)) + r"$\theta_c$"
        plt.plot(theta_r, mdrc, linewidth=1.5, linestyle=line_style[i], label=label)

    plt.xlabel(r'$\theta_r$ [rad]')
    plt.ylabel('mdrc')
    plt.ylim(bottom=0)
    plt.axvline(x=thetac, linewidth=0.5, color="black")
    plt.legend(loc='center left', title=r"$\theta_i$", bbox_to_anchor=(1, 0.5))
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################
    
    plt.figure(figsize=(3.375, 3.0)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "MDRC p-p Polarization"
    mdrc = np.zeros(shape=points)
    line_style = ['-.', '--', '-', '--', '-.']

    for i in range(len(theta_inc)):
        mdrc[:] = MDRC_list[0, 0, i, :]
        label = str(np.round(theta_inc[i]/thetac, 2)) + r"$\theta_c$"
        plt.plot(theta_r, mdrc, linewidth=1.5, linestyle=line_style[i], label=label)

    plt.xlabel(r'$\theta_r$ [rad]')
    plt.ylabel('mdrc')
    plt.ylim(bottom=0)
    plt.axvline(x=thetac, linewidth=0.5, color="black")
    plt.legend(loc='center left', title=r"$\theta_i$", bbox_to_anchor=(1, 0.5))
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    plt.figure(figsize=(3.375, 3.0)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "First Derivative of MDRC s-s Polarization"
    dmdrc = np.zeros(shape=points)
    line_style = ['-.', '--', '-', '--', '-.']

    for i in range(len(theta_inc)):
        dmdrc[:] = dMDRC_list[1, 1, i, :]
        label = str(np.round(theta_inc[i]/thetac, 2)) + r"$\theta_c$"
        plt.plot(theta_r, dmdrc, linewidth=1.5, linestyle=line_style[i], label=label)

    plt.xlabel(r'$\theta_r$ [rad]')
    plt.ylabel(r'd(mdrc)/d$\theta_r$')
    plt.axvline(x=thetac, linewidth=0.5, color="black")
    plt.legend(loc='center left', title=r"$\theta_i$", bbox_to_anchor=(1, 0.5))
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    plt.figure(figsize=(3.375, 3.0)) 
    plt.rcParams.update({
    'axes.titlesize': 10,
    'axes.labelsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10})
    plt.tight_layout()

    title = "First Derivative of MDRC p-p Polarization"
    dmdrc = np.zeros(shape=points)
    line_style = ['-.', '--', '-', '--', '-.']

    for i in range(len(theta_inc)):
        dmdrc[:] = dMDRC_list[0, 0, i, :]
        label = str(np.round(theta_inc[i]/thetac, 2)) + r"$\theta_c$"
        plt.plot(theta_r, dmdrc, linewidth=1.5, linestyle=line_style[i], label=label)

    plt.xlabel(r'$\theta_r$ [rad]')
    plt.ylabel(r'd(mdrc)/d$\theta_r$')
    plt.axvline(x=thetac, linewidth=0.5, color="black")
    plt.legend(loc='center left', title=r"$\theta_i$", bbox_to_anchor=(1, 0.5))
    plt.grid(True)

    plot_title = title + ".pdf"
    plot_output = os.path.join(outdir, plot_title)
    plt.savefig(plot_output)
    plt.close()

    #################################################################################################################################

    date_finish = datetime.now()
    finish = date_finish.strftime("%d-%m-%Y_%H.%M.%S")
    with open(summary_output, 'a') as file:
        file.write("\n")
        file.write("Date finished : " + finish + "\n")